In [2]:
from PIL import Image, ImageOps
import numpy as np
import sys
import matplotlib.pyplot as plt

In [3]:
SAVE_FOLDER = '.'
PATH_TO_DATA = '../data/'

In [4]:
sys.path.append('../src/')
sys.path.append('../src/lib/data_loaders/')

In [5]:
from Sat2RadModule import Sat2RadDataModule
from DatasetDistributor import DatasetDistributor
from DatasetType import DatasetType

In [6]:
module = Sat2RadDataModule(data_dir=PATH_TO_DATA, batch_size=1, splits={"train": 0.6, "val": 0.1, "test": 0.1}, dataset_type=1)
module.setup(None)

1


                                                 Partitioned Files                                                 
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃                ┃                 ┃                ┃                 ┃            ┃          Files ┃             ┃
┃ Start Date Sat ┃    End Date Sat ┃ Start Date Rad ┃    End Date Rad ┃ Partition  ┃      Satellite ┃ Files Radar ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│     2023-03-01 │      2023-04-05 │     2023-03-01 │      2023-04-05 │ All        │           3331 │       10348 │
│       23:42:00 │        22:42:00 │       00:05:00 │        22:55:00 │            │                │             │
│     2023-03-01 │      2023-03-22 │     2023-03-01 │      2023-03-22 │ Training   │           1998 │        6206 │
│       23:42:00 │        21:27:00 │       00:05:00 │        13:25:00 │            │                │             │
│     2023-03-22 │      2023-03-26 │     2023-03-22 │      2023-03-26 │ Validation │            333 │        1034 │
│       21:27:00 │        08:42:00 │       13:25:00 │        03:50:00 │            │                │             │
│     2023-03-26 │      2023-03-29 │     2023-03-26 │      2023-03-29 │ Testing    │            333 │        1034 │
│       08:42:00 │        20:27:00 │       03:50:00 │        18:05:00 │            │                │             │
└────────────────┴─────────────────┴────────────────┴─────────────────┴────────────┴────────────────┴─────────────┘

using the dataset: <class 'DatasetSlidingWindow.Sat2RadDatasetSlidingWindow'>

In [7]:
train = module.train_dataloader()

In [8]:
test = module.test_dataloader()

In [9]:
val = module.val_dataloader()

In [10]:
gt = [data[1].detach().numpy() for data in iter(val)]

In [11]:
gt = np.array(gt)

In [12]:
gt.shape

(300, 1, 1, 256, 256)

In [13]:
gt = gt.reshape(-1, 256, 256)

In [14]:
gt.shape

(300, 256, 256)

In [15]:
satellite = [data[0].detach().numpy() for data in iter(val)]

In [16]:
satellite = np.array(satellite)

In [17]:
satelliteog = satellite.reshape((-1, 11, 256, 256))

In [18]:
satellite = satelliteog[:, 8, :, :]

In [19]:
def make_gif(array: np.ndarray, pred=True, name: str = 'gifyboy'):
    # imgs = np.random.randint(0, 255, (100, 50, 50, 3), dtype=np.uint8)
    array = array * 255
    imgs = [Image.fromarray(img) for img in array]
    # duration is the number of milliseconds between frames; this is 40 frames per second
    imgs[0].save(
        f"{SAVE_FOLDER}{name}_vid.gif",
        save_all=True,
        append_images=imgs[1:],
        duration=100,
        loop=0,
    )

In [22]:
def make_gif_with_color(array: np.ndarray, pred=True, name: str = 'gifyboy'):
    array = array * 255
    imgs = [Image.fromarray(img).convert("L") for img in array]
    imgs = [ImageOps.colorize(img, black=(24, 24, 27), white='white') for img in imgs]
    # duration is the number of milliseconds between frames; this is 40 frames per second
    imgs[0].save(
        f"{SAVE_FOLDER}{name}_vid.gif",
        save_all=True,
        append_images=imgs[1:],
        duration=100,
        loop=0,
    )

In [23]:
make_gif_with_color(gt)

In [19]:
make_gif(gt)

TypeError: Cannot handle this data type: (1, 1), <i8

In [61]:
make_gif_with_color(satellite)

In [28]:
make_gif(satellite, name="satellite")

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe9d1739a20>
Traceback (most recent call last):
  File "/home/mark/miniconda3/envs/torch/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/mark/miniconda3/envs/torch/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1442, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/mark/miniconda3/envs/torch/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/home/mark/miniconda3/envs/torch/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/home/mark/miniconda3/envs/torch/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/home/mark/miniconda3/envs/torch/lib/python3.10/selectors.py", line 416, in select
    fd_event_

In [31]:
gtSat = [data[0].detach().numpy() for data in iter(train)]

In [33]:
gtSat = np.array(gtSat)

In [34]:
gtSat.shape

(9, 1, 5, 11, 166, 134)

In [36]:
gtSat = gtSat.reshape(-1, 11, 166, 134)

In [41]:
gtSat.shape

(45, 11, 166, 134)

In [42]:
imfs = gtSat[0]

In [43]:
make_gif(imfs)